# Deploy to Triton Inference Server locally

description: (preview) deploy an image classification model trained on densenet locally via Triton

Please note that this Public Preview release is subject to the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

In [10]:
!pip install nvidia-pyindex
!pip install --upgrade tritonclient
!curl -L https://aka.ms/azureml-core-1.19.a1 --output azureml_core-1.19.0a1-py3-none-any.whl
!pip install azureml_core-1.19.0a1-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Requirement already up-to-date: tritonclient in /home/gopalv/miniconda3/envs/azureml/lib/python3.7/site-packages (2.4.0)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [11]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='Inference-PM-AML-Workspace', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='Inference-PM')

## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [19]:
import os
import sys
from pathlib import Path
from src.model_utils import download_triton_models, delete_triton_models

prefix = Path(".")
download_triton_models(prefix)

successfully downloaded model: densenet_onnx
successfully downloaded model: bidaf-9


## Register model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.

In [13]:
from azureml.core.model import Model

model_path = "models"

model = Model.register(
    model_path=model_path,
    model_name="densenet-onnx-example",
    tags={"area": "Image classification", "type": "classification"},
    description="Image classification trained on Imagenet Dataset",
    workspace=ws,
)

print(model)

Registering model densenet-onnx-example
Model(workspace=Workspace.create(name='Inference-PM-AML-Workspace', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='Inference-PM'), name=densenet-onnx-example, id=densenet-onnx-example:9, version=9, tags={'area': 'Image classification', 'type': 'classification'}, properties={})


## Deploy webservice

In this case we deploy to the local compute, but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli).


In [15]:
from azureml.core.webservice import LocalWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from random import randint

service_name = "triton-densenet-onnx-local" + str(randint(10000, 99999))


env = Environment("triton-tutorial")
env.docker.base_image = None
env.docker.base_dockerfile = "Dockerfile"
env.python.user_managed_dependencies = True
env.python.interpreter_path = "/opt/miniconda/bin/python"

env.environment_variables["WORKER_COUNT"] = "1"

inference_config = InferenceConfig(
    # this entry script is where we dispatch a call to the Triton server
    source_directory="src",
    entry_script="score_densenet.py",
    environment=env,
)

config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'
Generating Docker build context.
Package creation Succeeded
Logging into Docker registry inferencepmaef584480.azurecr.io
Logging into Docker registry inferencepmaef584480.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM inferencepmaef584480.azurecr.io/azureml/azureml_0d59db2c4b282d80ea50460afb86a2ee
 ---> 8ada067971f0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 785d7c027f7b
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjkyYzc2YTJmLTBlMWMtNDIxNi1iNjVlLWFiZjdhM2YzNGMxZSIsInJlc291cmNlR3JvdXBOYW1lIjoiaW5mZXJlbmNlLXBtIiwiYWNjb3VudE5hbWUiOiJpbmZlcmVuY2UtcG0tYW1sLXdvcmtzcGFjZSIsIndvcmtzcGFjZUlkIjoiODI0NDI5MGEtMzUwNi00MDEyLTgxNjAtOGQ4NjcwMzMzZTE5In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Ru

In [16]:
print(service.get_logs())

2020-11-19T06:50:25,842226200+00:00 - gunicorn/run 
2020-11-19T06:50:25,842401500+00:00 - rsyslog/run 
2020-11-19T06:50:25,842413500+00:00 - triton/run 

== Triton Inference Server ==

NVIDIA Release 20.08 (build 15533555)

Copyright (c) 2018-2020, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.
2020-11-19T06:50:25,869858400+00:00 - Waiting for Triton server to get ready ...
find: File system loop detected; ‘/usr/bin/X11’ is part of the same file system loop as ‘/usr/bin’.

   Use 'nvidia-docker run' to start this container; see
   https://github.com/NVIDIA/nvidia-docker/wiki/nvidia-docker .

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-

## Test the webservice

In [17]:
import requests

headers = {"Content-Type": "application/octet-stream"}

test_sample = requests.get("https://aka.ms/peacock-pic", allow_redirects=True).content
resp = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(resp.text)

unexpected shape for input 'data_0' for model 'densenet_onnx'. Expected [3,224,224], got [1,3,224,224]


## Delete the webservice and the downloaded model

In [18]:
service.delete()
delete_triton_models(prefix)

Container has been successfully cleaned up.
model: densenet_onnx was already deleted
model: bidaf-9 was already deleted


# Next steps

Try changing the deployment configuration to [deploy to Azure Kubernetes Service](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python) for higher availability and better scalability.